In [ ]:
!pip install keyboard

In [6]:
import pyaudio
import opensmile
import speech_recognition as sr
import numpy as np

# Initialiser la capture audio avec PyAudio
audio_format = pyaudio.paInt16  # Format de l'audio
channels = 1  # Mono
rate = 16000  # Taux d'échantillonnage
chunk_size = 1024  # Taille des morceaux de l'audio

# Initialiser OpenSMILE pour l'extraction des caractéristiques
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals
)

# Fonction pour capturer et analyser l'audio
def capture_audio():
    # Initialiser le flux d'entrée audio
    p = pyaudio.PyAudio()
    stream = p.open(format=audio_format, channels=channels,
                    rate=rate, input=True, frames_per_buffer=chunk_size)

    print("Appuyez sur 'Entrée' pour commencer à parler, puis 'Entrée' à nouveau pour arrêter...")

    frames = []
    recording = False

    while True:
        # Attente de l'appui sur 'Entrée' pour commencer l'enregistrement
        if not recording:
            input()  # Attendre l'appui sur la touche "Entrée"
            print("Enregistrement démarré...")
            recording = True

        if recording:
            data = stream.read(chunk_size)
            frames.append(data)

        # Attendre l'appui sur 'Entrée' pour arrêter l'enregistrement
        if recording:
            print("Appuyez à nouveau sur 'Entrée' pour arrêter l'enregistrement...")
            input()  # Attendre l'appui sur la touche "Entrée"
            print("Enregistrement arrêté.")
            break  # Arrêter l'enregistrement

    # Convertir les frames capturées en un tableau NumPy
    audio_data = np.frombuffer(b''.join(frames), dtype=np.int16)

    # Arrêter et fermer le flux audio
    stream.stop_stream()
    stream.close()

    print("Audio capturé en mémoire.")

    # Extraire les caractéristiques audio avec OpenSMILE
    features = smile.process_signal(audio_data.astype(np.float32), rate)

    # Afficher les caractéristiques extraites
    print("Caractéristiques audio extraites :")
    print(features)

    # Retranscription audio en texte avec SpeechRecognition
    recognizer = sr.Recognizer()

    # Utiliser SpeechRecognition pour transcrire l'audio capturé
    audio = sr.AudioData(audio_data.tobytes(), rate, 2)  # Convertir l'audio numpy en AudioData

    try:
        # Utiliser Google Speech Recognition pour la transcription
        text = recognizer.recognize_google(audio)
        print("Texte retranscrit :")
        print(text)
    except sr.UnknownValueError:
        print("Impossible de comprendre l'audio.")
    except sr.RequestError as e:
        print(f"Erreur avec le service de reconnaissance vocale; {e}")

# Appeler la fonction pour commencer à capturer
capture_audio()


Appuyez sur 'Entrée' pour commencer à parler, puis 'Entrée' à nouveau pour arrêter...


KeyboardInterrupt: Interrupted by user

In [10]:
import pyaudio
import wave
import threading
import speech_recognition as sr
import numpy as np

# Paramètres d'enregistrement
audio_format = pyaudio.paInt16
channels = 1
rate = 16000
chunk_size = 1024

def record_audio(stream, frames, stop_event):
    while not stop_event.is_set():
        data = stream.read(chunk_size)
        frames.append(data)

def main():
    # Initialiser PyAudio et le flux d'enregistrement
    p = pyaudio.PyAudio()
    stream = p.open(format=audio_format, channels=channels,
                    rate=rate, input=True, frames_per_buffer=chunk_size)
    frames = []
    stop_event = threading.Event()

    input("Appuyez sur 'Entrée' pour commencer l'enregistrement...")
    print("Enregistrement démarré...")
    record_thread = threading.Thread(target=record_audio, args=(stream, frames, stop_event))
    record_thread.start()
    input("Appuyez sur 'Entrée' pour arrêter l'enregistrement...")
    stop_event.set()
    record_thread.join()

    # Arrêter et fermer le flux
    stream.stop_stream()
    stream.close()
    p.terminate()

    # Sauvegarder l'audio dans un fichier WAV
    wav_file = "captured_audio.wav"
    with wave.open(wav_file, 'wb') as wf:
        wf.setnchannels(channels)
        wf.setsampwidth(p.get_sample_size(audio_format))
        wf.setframerate(rate)
        wf.writeframes(b''.join(frames))
    print(f"Audio sauvegardé dans '{wav_file}'.")

    # Retranscription du fichier WAV
    recognizer = sr.Recognizer()
    with sr.AudioFile(wav_file) as source:
        audio = recognizer.record(source)
    try:
        print("Transcription en cours...")
        text = recognizer.recognize_google(audio, language="fr-FR")
        print(f"Texte retranscrit : {text}")
    except sr.UnknownValueError:
        print("Impossible de comprendre l'audio.")
    except sr.RequestError as e:
        print(f"Erreur avec le service de reconnaissance vocale : {e}")

if __name__ == "__main__":
    main()


Appuyez sur 'Entrée' pour commencer l'enregistrement...
Enregistrement démarré...
Appuyez sur 'Entrée' pour arrêter l'enregistrement...
Audio sauvegardé dans 'captured_audio.wav'.
Transcription en cours...
Texte retranscrit : bonjour ceci est un test de culture pour vérifier le fonctionnement du système de reconnaissance vocale


Bonjour, ceci est un test de capture audio pour vérifier le fonctionnement du système de reconnaissance vocale.

In [14]:
import speech_recognition as sr
import time

def callback(recognizer, audio):
    try:
        text = recognizer.recognize_google(audio, language="fr-FR")
        print("Texte retranscrit :", text)
    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

print("Démarrage de la transcription en temps réel...")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Garder le programme actif pour la transcription continue
try:
    while True:
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")


Démarrage de la transcription en temps réel...
Texte retranscrit : bonjour
Texte retranscrit : allô
Texte retranscrit : pour ceci est un test de capture audio pour vérifier le fonctionnement du système de reconnaissance vocale
Texte retranscrit : bonjour
Texte retranscrit : que c'est un test de
Texte retranscrit : bonjour ceci est un test de capture audio pour vérifier le fonctionnement du système de reconnaissance
Texte retranscrit : bonjour ceci est un test de capture audio pour vérifier le fonctionnement du système de reconnaissance
Texte retranscrit : OK
Transcription arrêtée.


In [15]:
import speech_recognition as sr
import time

def callback(recognizer, audio):
    try:
        text = recognizer.recognize_google(audio, language="fr-FR")
        print("Texte retranscrit :", text)
    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 1.5  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Garder le programme actif pour la transcription continue
try:
    while True:
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")

Démarrage de la transcription en temps réel...
Texte retranscrit : bonjour
Texte retranscrit : bonjour ceci
Texte retranscrit : un test
Texte retranscrit : de capture audio pour vérifier le fonctionnement du système de reconnaissance
Transcription arrêtée.


In [18]:
import speech_recognition as sr
import time

def callback(recognizer, audio):
    try:
        # Reconnaître l'audio
        text = recognizer.recognize_google(audio, language="fr-FR")
        print("Texte retranscrit :", text)

        # Si la phrase "programme arrête toi" est détectée, arrêter l'écoute
        if "stoppe stoppe stoppe" in text.lower():
            print("Phrase 'programme arrête toi' détectée. Arrêt du programme.")
            stop_listening(wait_for_stop=False)  # Arrêter l'écoute en arrière-plan

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 1.5  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Garder le programme actif pour la transcription continue
try:
    while True:
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")


Démarrage de la transcription en temps réel...
Texte retranscrit : allô
Texte retranscrit : ceci est un test de capture audio
Texte retranscrit : stoppe stoppe
Texte retranscrit : stoppe stoppe
Texte retranscrit : stoppe stoppe stoppe stoppe stoppe stoppe stoppe stoppe
Phrase 'programme arrête toi' détectée. Arrêt du programme.
Transcription arrêtée.


In [28]:
import speech_recognition as sr
import time
import threading

def callback(recognizer, audio):
    try:
        # Reconnaître l'audio
        text = recognizer.recognize_google(audio, language="fr-FR")
        print("Texte retranscrit :", text)

        # Si la phrase "stoppe stoppe stoppe" est détectée, arrêter l'écoute
        if "stoppe stoppe stoppe" in text.lower():
            print("Phrase 'stoppe stoppe stoppe' détectée. Arrêt du programme.")
            # Utiliser un événement global pour arrêter l'écoute
            stop_listening(wait_for_stop=False)

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 1.5  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Garder le programme actif pour la transcription continue
try:
    while True:
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")


Démarrage de la transcription en temps réel...
Texte retranscrit : allô
Texte retranscrit : ceci est un test
Transcription arrêtée.


In [25]:
!pip install langdetect

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     --- ---------------------------------- 102.4/981.5 kB 3.0 MB/s eta 0:00:01
     ---------- --------------------------- 266.2/981.5 kB 2.8 MB/s eta 0:00:01
     --------------- ---------------------- 389.1/981.5 kB 2.4 MB/s eta 0:00:01
     ------------------- ------------------ 501.8/981.5 kB 2.6 MB/s eta 0:00:01
     ----------------------- -------------- 614.4/981.5 kB 2.6 MB/s eta 0:00:01
     ------------------------------ ------- 798.7/981.5 kB 2.7 MB/s eta 0:00:01
     ---------------------------------- --- 880.6/981.5 kB 2.5 MB/s eta 0:00:01
     -------------------------------------  972.8/981.5 kB 2.6 MB/s eta 0:00:01
     -------------------------------------  972.8/981.5 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langd

In [27]:
import speech_recognition as sr
import time
from langdetect import detect  # Installer langdetect avec 'pip install langdetect'

# Dictionnaire pour les langues supportées
supported_languages = {
    "fr": "fr-FR",
    "en": "en-US",
    "es": "es-ES",
    "de": "de-DE",
    "it": "it-IT",
    "pt": "pt-PT",
    "ru": "ru-RU",
    "ja": "ja-JP",
    "zh": "zh-CN",
    # Ajouter plus de langues si nécessaire
}

def callback(recognizer, audio):
    try:
        # Reconnaître l'audio
        text = recognizer.recognize_google(audio)
        print("Texte retranscrit :", text)

        # Détecter la langue du texte
        detected_language = detect(text)
        print(f"Langue détectée : {detected_language}")

        # Vérifier si la langue détectée est supportée
        if detected_language in supported_languages:
            language_code = supported_languages[detected_language]
        else:
            # Par défaut, utiliser l'anglais si la langue n'est pas supportée
            language_code = "en-US"

        # Effectuer une nouvelle reconnaissance avec la langue détectée
        text = recognizer.recognize_google(audio, language=language_code)
        print(f"Texte retranscrit dans la langue détectée ({language_code}):", text)

        # Phrase de fin
        if "programme arrête toi" in text.lower():
            print("Phrase 'programme arrête toi' détectée. Arrêt du programme.")
            stop_listening(wait_for_stop=False)  # Arrêter l'écoute en arrière-plan

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 0.8  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Garder le programme actif pour la transcription continue
try:
    while True:
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")


Démarrage de la transcription en temps réel...
Transcription arrêtée.


In [30]:
import speech_recognition as sr
import time
import tkinter as tk
from tkinter import messagebox

# Dictionnaire pour les langues supportées
supported_languages = {
    "français": "fr-FR",
    "english": "en-US",
    "español": "es-ES",
    "deutsch": "de-DE",
    "italiano": "it-IT",
    "português": "pt-PT",
    "русский": "ru-RU",
    "日本語": "ja-JP",
    "中文": "zh-CN",
}

# Langue par défaut
current_language = "fr-FR"

def callback(recognizer, audio):
    global current_language
    try:
        # Reconnaître l'audio avec la langue actuelle
        text = recognizer.recognize_google(audio, language=current_language)
        print(f"Texte retranscrit ({current_language}):", text)

        # Si la phrase "stoppe stoppe stoppe" est détectée, arrêter l'écoute
        if "stoppe stoppe stoppe" in text.lower():
            print("Phrase 'stoppe stoppe stoppe' détectée. Arrêt du programme.")
            stop_listening(wait_for_stop=False)  # Arrêter l'écoute en arrière-plan

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

def change_language(language):
    global current_language
    if language in supported_languages:
        current_language = supported_languages[language]
        print(f"Langue changée en {language}.")
    else:
        print(f"Langue '{language}' non supportée.")

def open_language_window():
    # Créer la fenêtre de changement de langue
    window = tk.Tk()
    window.title("Change Language")

    # Ajouter un bouton pour chaque langue
    for language in supported_languages.keys():
        button = tk.Button(window, text=language, command=lambda lang=language: change_language(lang))
        button.pack(pady=5)

    # Démarrer la fenêtre
    window.mainloop()

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 0.8  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Ouvrir la fenêtre pour changer de langue (dans un thread séparé pour ne pas bloquer l'écoute)
language_thread = threading.Thread(target=open_language_window)
language_thread.daemon = True
language_thread.start()

# Garder le programme actif pour la transcription continue
try:
    while True:
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")


Démarrage de la transcription en temps réel...
Texte retranscrit (fr-FR): bonjour
Texte retranscrit (fr-FR): ceci est un test
Langue changée en english.
Texte retranscrit (en-US): hello
Texte retranscrit (en-US): stop
Texte retranscrit (en-US): stop stop
Texte retranscrit (en-US): stop stop stop
Transcription arrêtée.


In [32]:
import speech_recognition as sr
import time
import tkinter as tk
import threading

# Dictionnaire pour les langues supportées
supported_languages = {
    "français": "fr-FR",
    "english": "en-US",
    "español": "es-ES",
    "deutsch": "de-DE",
    "italiano": "it-IT",
    "português": "pt-PT",
    "русский": "ru-RU",
    "日本語": "ja-JP",
    "中文": "zh-CN",
}

# Dictionnaire des phrases d'arrêt selon la langue
stop_phrases = {
    "fr-FR": "stoppe stoppe",  # français
    "en-US": "stop stop",        # anglais
    "es-ES": "para para",       # espagnol
    "de-DE": "stopp stopp",     # allemand
    "it-IT": "ferma ferma",     # italien
    "pt-PT": "pare pare",       # portugais
    "ru-RU": "стоп стоп",       # russe
    "ja-JP": "ストップ ストップ",  # japonais
    "zh-CN": "停止 停止",         # chinois
}

# Langue par défaut
current_language = "fr-FR"
stop_flag = False  # Flag pour arrêter le programme

def callback(recognizer, audio):
    global stop_flag, current_language
    try:
        # Reconnaître l'audio avec la langue actuelle
        text = recognizer.recognize_google(audio, language=current_language)
        print(f"Texte retranscrit ({current_language}):", text)

        # Vérifier la phrase d'arrêt
        if stop_phrases.get(current_language) and stop_phrases[current_language] in text.lower():
            print(f"Phrase '{stop_phrases[current_language]}' détectée. Arrêt du programme.")
            stop_flag = True  # Activer le flag pour arrêter le programme
            stop_listening(wait_for_stop=False)  # Arrêter l'écoute en arrière-plan

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

def change_language(language):
    global current_language
    if language in supported_languages:
        current_language = supported_languages[language]
        print(f"Langue changée en {language}.")
    else:
        print(f"Langue '{language}' non supportée.")

def open_language_window():
    # Créer la fenêtre de changement de langue
    window = tk.Tk()
    window.title("Change Language")

    # Ajouter un bouton pour chaque langue
    for language in supported_languages.keys():
        button = tk.Button(window, text=language, command=lambda lang=language: change_language(lang))
        button.pack(pady=5)

    # Démarrer la fenêtre
    window.mainloop()

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 0.8  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Ouvrir la fenêtre pour changer de langue (dans un thread séparé pour ne pas bloquer l'écoute)
language_thread = threading.Thread(target=open_language_window)
language_thread.daemon = True
language_thread.start()

# Garder le programme actif pour la transcription continue
try:
    while not stop_flag:  # Boucle qui s'arrête si stop_flag est activé
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")


Démarrage de la transcription en temps réel...
Texte retranscrit (fr-FR): allô
Texte retranscrit (fr-FR): bonjour
Langue changée en english.
Texte retranscrit (en-US): hello
Langue changée en español.
Texte retranscrit (es-ES): fue un
Texte retranscrit (es-ES): hola
Texte retranscrit (es-ES): uff pam pam
Texte retranscrit (es-ES): para para para
Phrase 'para para para' détectée. Arrêt du programme.


In [33]:
import speech_recognition as sr
import time
import tkinter as tk
import threading

# Dictionnaire pour les langues supportées
supported_languages = {
    "français": "fr-FR",
    "english": "en-US",
    "español": "es-ES",
    "deutsch": "de-DE",
    "italiano": "it-IT",
    "português": "pt-PT",
    "русский": "ru-RU",
    "日本語": "ja-JP",
    "中文": "zh-CN",
    "العربية": "ar-SA",  # Arabe ajouté
}

# Dictionnaire des phrases d'arrêt selon la langue
stop_phrases = {
    "fr-FR": "stoppe stoppe",  # français
    "en-US": "stop stop",        # anglais
    "es-ES": "para para",       # espagnol
    "de-DE": "stopp stopp",     # allemand
    "it-IT": "ferma ferma",     # italien
    "pt-PT": "pare pare",       # portugais
    "ru-RU": "стоп стоп",       # russe
    "ja-JP": "ストップ ストップ",  # japonais
    "zh-CN": "停止 停止",         # chinois
    "ar-SA": "توقف توقف",         # arabe
}

# Langue par défaut
current_language = "fr-FR"
stop_flag = False  # Flag pour arrêter le programme

def callback(recognizer, audio):
    global stop_flag, current_language, window
    try:
        # Reconnaître l'audio avec la langue actuelle
        text = recognizer.recognize_google(audio, language=current_language)
        print(f"Texte retranscrit ({current_language}):", text)

        # Vérifier la phrase d'arrêt
        if stop_phrases.get(current_language) and stop_phrases[current_language] in text.lower():
            print(f"Phrase '{stop_phrases[current_language]}' détectée. Arrêt du programme.")
            stop_flag = True  # Activer le flag pour arrêter le programme
            stop_listening(wait_for_stop=False)  # Arrêter l'écoute en arrière-plan
            if window:
                window.quit()  # Fermer la fenêtre Tkinter

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

def change_language(language):
    global current_language
    if language in supported_languages:
        current_language = supported_languages[language]
        print(f"Langue changée en {language}.")
        if window:
            window.quit()  # Fermer la fenêtre quand une langue est changée
    else:
        print(f"Langue '{language}' non supportée.")

def open_language_window():
    global window
    # Créer la fenêtre de changement de langue
    window = tk.Tk()
    window.title("Change Language")

    # Ajouter un bouton pour chaque langue et afficher les phrases d'arrêt
    for language, code in supported_languages.items():
        stop_phrase = stop_phrases.get(code, "")
        button_text = f"{language} - Phrase d'arrêt : {stop_phrase}"
        button = tk.Button(window, text=button_text, command=lambda lang=language: change_language(lang))
        button.pack(pady=5)

    # Démarrer la fenêtre
    window.mainloop()

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 0.8  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Ouvrir la fenêtre pour changer de langue (dans un thread séparé pour ne pas bloquer l'écoute)
language_thread = threading.Thread(target=open_language_window)
language_thread.daemon = True
language_thread.start()

# Garder le programme actif pour la transcription continue
try:
    while not stop_flag:  # Boucle qui s'arrête si stop_flag est activé
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")
    if window:
        window.quit()  # Fermer la fenêtre Tkinter quand l'utilisateur interrompt le programme


Démarrage de la transcription en temps réel...
Langue changée en français.
Transcription arrêtée.


In [4]:
import speech_recognition as sr
import time
import tkinter as tk
import threading

# Dictionnaire pour les langues supportées
supported_languages = {
    "français": "fr-FR",
    "english": "en-US",
    "español": "es-ES",
    "deutsch": "de-DE",
    "italiano": "it-IT",
    "português": "pt-PT",
    "русский": "ru-RU",
    "日本語": "ja-JP",
    "中文": "zh-CN",
    "العربية": "ar-SA",
}

# Dictionnaire des phrases d'arrêt selon la langue
stop_phrases = {
    "fr-FR": "stoppe stoppe",  # français
    "en-US": "stop stop",        # anglais
    "es-ES": "para para",       # espagnol
    "de-DE": "stopp stopp",     # allemand
    "it-IT": "ferma ferma",     # italien
    "pt-PT": "pare pare",       # portugais
    "ru-RU": "стоп стоп",       # russe
    "ja-JP": "ストップ ストップ",  # japonais
    "zh-CN": "停止 停止",         # chinois
    "ar-SA": "توقف توقف",         # arabe
}

# Langue par défaut
current_language = "fr-FR"
stop_flag = False  # Flag pour arrêter le programme

def callback(recognizer, audio):
    global stop_flag, current_language, window
    try:
        # Reconnaître l'audio avec la langue actuelle
        text = recognizer.recognize_google(audio, language=current_language)
        print(f"Texte retranscrit ({current_language}):", text)

        # Vérifier la phrase d'arrêt
        if stop_phrases.get(current_language) and stop_phrases[current_language] in text.lower():
            print(f"Phrase '{stop_phrases[current_language]}' détectée. Arrêt du programme.")
            stop_flag = True  # Activer le flag pour arrêter le programme
            stop_listening(wait_for_stop=False)  # Arrêter l'écoute en arrière-plan
            if window:
                window.quit()  # Fermer la fenêtre Tkinter si l'arrêt est détecté

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

def change_language(language):
    global current_language
    if language in supported_languages:
        current_language = supported_languages[language]
        print(f"Langue changée en {language}.")
        # Ne fermer la fenêtre, mais simplement changer la langue
        update_buttons()

def update_buttons():
    # Mettre à jour les boutons de la fenêtre pour afficher la langue actuelle et la phrase d'arrêt
    for widget in window.winfo_children():
        widget.destroy()  # Supprimer les anciens boutons

    # Ajouter un bouton pour chaque langue et afficher les phrases d'arrêt
    for language, code in supported_languages.items():
        stop_phrase = stop_phrases.get(code, "")
        button_text = f"{language} - Phrase d'arrêt : {stop_phrase}"
        button = tk.Button(window, text=button_text, command=lambda lang=language: change_language(lang))
        button.pack(pady=5)

def open_language_window():
    global window
    # Créer la fenêtre de changement de langue
    window = tk.Tk()
    window.title("Change Language")

    # Ajouter un bouton pour chaque langue et afficher les phrases d'arrêt
    update_buttons()

    # Démarrer la fenêtre
    window.mainloop()

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 1  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")
print(f"Langue par défaut : {current_language}")
print(f"Commencez à parler")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Ouvrir la fenêtre pour changer de langue (dans un thread séparé pour ne pas bloquer l'écoute)
language_thread = threading.Thread(target=open_language_window)
language_thread.daemon = True
language_thread.start()

# Garder le programme actif pour la transcription continue
try:
    while not stop_flag:  # Boucle qui s'arrête si stop_flag est activé
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")
    if window:
        window.quit()  # Fermer la fenêtre Tkinter quand l'utilisateur interrompt le programme


Démarrage de la transcription en temps réel...
Langue par défaut : fr-FR
Commencez à parler
Texte retranscrit (fr-FR): stoppe stoppe
Phrase 'stoppe stoppe' détectée. Arrêt du programme.


Bonjour, ceci est un test de capture audio pour vérifier le fonctionnement du système de reconnaissance vocale.

In [2]:
import speech_recognition as sr
import time
import tkinter as tk
import threading

# Dictionnaire pour les langues supportées
supported_languages = {
    "français": "fr-FR",
    "english": "en-US",
    "español": "es-ES",
    "deutsch": "de-DE",
    "italiano": "it-IT",
    "português": "pt-PT",
    "русский": "ru-RU",
    "日本語": "ja-JP",
    "中文": "zh-CN",
    "العربية": "ar-SA",
}

# Dictionnaire des phrases d'arrêt selon la langue
stop_phrases = {
    "fr-FR": "stoppe stoppe",  # français
    "en-US": "stop stop",        # anglais
    "es-ES": "para para",       # espagnol
    "de-DE": "stopp stopp",     # allemand
    "it-IT": "ferma ferma",     # italien
    "pt-PT": "pare pare",       # portugais
    "ru-RU": "стоп стоп",       # russe
    "ja-JP": "ストップ ストップ",  # japonais
    "zh-CN": "停止 停止",         # chinois
    "ar-SA": "توقف توقف",         # arabe
}

# Langue par défaut
current_language = "fr-FR"
stop_flag = False  # Flag pour arrêter le programme

def callback(recognizer, audio):
    global stop_flag, current_language, window
    try:
        # Reconnaître l'audio avec la langue actuelle
        text = recognizer.recognize_google(audio, language=current_language)
        print(f"Texte retranscrit ({current_language}):", text)

        # Vérifier la phrase d'arrêt
        if stop_phrases.get(current_language) and stop_phrases[current_language] in text.lower():
            print(f"Phrase '{stop_phrases[current_language]}' détectée. Arrêt du programme.")
            stop_flag = True  # Activer le flag pour arrêter le programme
            stop_listening(wait_for_stop=False)  # Arrêter l'écoute en arrière-plan
            if window:
                window.quit()  # Fermer la fenêtre Tkinter si l'arrêt est détecté

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

def change_language(language):
    global current_language
    if language in supported_languages:
        current_language = supported_languages[language]
        print(f"Langue changée en {language}.")
        # Ne fermer la fenêtre, mais simplement changer la langue
        update_buttons()

def update_buttons():
    # Mettre à jour les boutons de la fenêtre pour afficher la langue actuelle et la phrase d'arrêt
    for widget in window.winfo_children():
        widget.destroy()  # Supprimer les anciens boutons

    # Ajouter un bouton pour chaque langue et afficher les phrases d'arrêt
    for language, code in supported_languages.items():
        stop_phrase = stop_phrases.get(code, "")
        button_text = f"{language} - Phrase d'arrêt : {stop_phrase}"
        button = tk.Button(window, text=button_text, command=lambda lang=language: change_language(lang))
        button.pack(pady=5)

def open_language_window():
    global window
    # Créer la fenêtre de changement de langue
    window = tk.Tk()
    window.title("Change Language")

    # Ajouter un bouton pour chaque langue et afficher les phrases d'arrêt
    update_buttons()

    # Placer la fenêtre au premier plan
    window.lift()  # Met la fenêtre Tkinter au premier plan
    window.attributes('-topmost', True)  # Assure que la fenêtre reste toujours au-dessus
    window.after(100, lambda: window.attributes('-topmost', False))  # Assure que la fenêtre ne reste pas constamment au-dessus

    # Démarrer la fenêtre
    window.mainloop()

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 1  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")
print(f"Langue par défaut : {current_language}")
print(f"Commencez à parler")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Ouvrir la fenêtre pour changer de langue (dans un thread séparé pour ne pas bloquer l'écoute)
language_thread = threading.Thread(target=open_language_window)
language_thread.daemon = True
language_thread.start()

# Garder le programme actif pour la transcription continue
try:
    while not stop_flag:  # Boucle qui s'arrête si stop_flag est activé
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")
    if window:
        window.quit()  # Fermer la fenêtre Tkinter quand l'utilisateur interrompt le programme


Démarrage de la transcription en temps réel...
Langue par défaut : fr-FR
Commencez à parler
Texte retranscrit (fr-FR): c'est compliqué c'est compliqué mais après il faut pas
Texte retranscrit (fr-FR): il avait pas des ennemis
Texte retranscrit (fr-FR): entreprise
Texte retranscrit (fr-FR): d'accord ok
Texte retranscrit (fr-FR): j'ai mis en place une place de données mais ça c'est
Texte retranscrit (fr-FR): le projet de synthèse
Texte retranscrit (fr-FR): j'imagine que tu fais des
Texte retranscrit (fr-FR): un petit peu même si je pense que tu fermes pas promis
Texte retranscrit (fr-FR): normal
Texte retranscrit (fr-FR): d'accord
Texte retranscrit (fr-FR): c'est ça exactement
Texte retranscrit (fr-FR): quand tu parles
Texte retranscrit (fr-FR): Pagnol allemand
Texte retranscrit (fr-FR): là tu
Texte retranscrit (fr-FR): allô
Langue changée en 中文.


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\boucceredj\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 75, in <lambda>
    button = tk.Button(window, text=button_text, command=lambda lang=language: change_language(lang))
                                                                               ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 64, in change_language
    update_buttons()
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 68, in update_buttons
    for widget in window.winfo_children():
                  ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\anaconda3\Lib\tkinter\__init__.py", line 1106, in winfo_children
    self.tk.call('winfo', 'children', self._w)):
    ^^^^^^^^^^^^^^^^^^^^^^^^

Texte retranscrit (zh-CN): 你好Hello
Texte retranscrit (zh-CN): 上面
Langue changée en español.


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\boucceredj\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 75, in <lambda>
    button = tk.Button(window, text=button_text, command=lambda lang=language: change_language(lang))
                                                                               ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 64, in change_language
    update_buttons()
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 68, in update_buttons
    for widget in window.winfo_children():
                  ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\anaconda3\Lib\tkinter\__init__.py", line 1106, in winfo_children
    self.tk.call('winfo', 'children', self._w)):
    ^^^^^^^^^^^^^^^^^^^^^^^^

Texte retranscrit (es-ES): cómo
Texte retranscrit (es-ES): gracias
Texte retranscrit (es-ES): cómo estás gracias gracias
Texte retranscrit (es-ES): hijo de tu madre
Texte retranscrit (es-ES): yo soy youtuber
Texte retranscrit (es-ES): después se me
Texte retranscrit (es-ES): sí de cómo se hace eso
Texte retranscrit (es-ES): pinche
Langue changée en العربية.
Texte retranscrit (ar-SA): mmm a todos por favor


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\boucceredj\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 75, in <lambda>
    button = tk.Button(window, text=button_text, command=lambda lang=language: change_language(lang))
                                                                               ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 64, in change_language
    update_buttons()
  File "C:\Users\boucceredj\AppData\Local\Temp\ipykernel_12484\3995342160.py", line 68, in update_buttons
    for widget in window.winfo_children():
                  ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\boucceredj\anaconda3\Lib\tkinter\__init__.py", line 1106, in winfo_children
    self.tk.call('winfo', 'children', self._w)):
    ^^^^^^^^^^^^^^^^^^^^^^^^

Texte retranscrit (ar-SA): اسمي ليسا
Texte retranscrit (ar-SA): مرحبا
Texte retranscrit (ar-SA): مرحبا
Texte retranscrit (ar-SA): ايتها الغبيه
Texte retranscrit (ar-SA): سلام عليكم سلام عليكم
Texte retranscrit (ar-SA): ايتها الغبيه
Texte retranscrit (ar-SA): الحمار
Texte retranscrit (ar-SA): بقره الى اللقاء
Texte retranscrit (ar-SA): معزه
Texte retranscrit (ar-SA): برتقالي
Transcription arrêtée.


In [6]:
import speech_recognition as sr
import time
import tkinter as tk
import threading
import opensmile
import numpy as np

# Dictionnaire pour les langues supportées
supported_languages = {
    "français": "fr-FR",
    "english": "en-US",
    "español": "es-ES",
    "deutsch": "de-DE",
    "italiano": "it-IT",
    "português": "pt-PT",
    "русский": "ru-RU",
    "日本語": "ja-JP",
    "中文": "zh-CN",
    "العربية": "ar-SA",
}

# Dictionnaire des phrases d'arrêt selon la langue
stop_phrases = {
    "fr-FR": "stoppe stoppe",  # français
    "en-US": "stop stop",        # anglais
    "es-ES": "para para",       # espagnol
    "de-DE": "stopp stopp",     # allemand
    "it-IT": "ferma ferma",     # italien
    "pt-PT": "pare pare",       # portugais
    "ru-RU": "стоп стоп",       # russe
    "ja-JP": "ストップ ストップ",  # japonais
    "zh-CN": "停止 停止",         # chinois
    "ar-SA": "توقف توقف",         # arabe
}

# Langue par défaut
current_language = "fr-FR"
stop_flag = False  # Flag pour arrêter le programme

# Initialiser OpenSMILE pour l'extraction des caractéristiques
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals
)

def callback(recognizer, audio):
    global stop_flag, current_language, window
    try:
        # Reconnaître l'audio avec la langue actuelle
        text = recognizer.recognize_google(audio, language=current_language)
        print(f"Texte retranscrit ({current_language}):", text)

        # Vérifier la phrase d'arrêt
        if stop_phrases.get(current_language) and stop_phrases[current_language] in text.lower():
            print(f"Phrase '{stop_phrases[current_language]}' détectée. Arrêt du programme.")
            stop_flag = True  # Activer le flag pour arrêter le programme
            stop_listening(wait_for_stop=False)  # Arrêter l'écoute en arrière-plan
            if window:
                window.quit()  # Fermer la fenêtre Tkinter si l'arrêt est détecté

        # Extraire les caractéristiques audio avec OpenSMILE
        audio_data = np.frombuffer(audio.get_raw_data(), dtype=np.int16)
        features = smile.process_signal(audio_data.astype(np.float32), 16000)
        print("Caractéristiques audio extraites :", features)

    except sr.UnknownValueError:
        pass  # Ne rien faire si l'audio est incompris
    except sr.RequestError as e:
        print("Erreur de service de reconnaissance vocale :", e)

def change_language(language):
    global current_language
    if language in supported_languages:
        current_language = supported_languages[language]
        print(f"Langue changée en {language}.")
        # Ne fermer la fenêtre, mais simplement changer la langue
        update_buttons()

def update_buttons():
    # Mettre à jour les boutons de la fenêtre pour afficher la langue actuelle et la phrase d'arrêt
    for widget in window.winfo_children():
        widget.destroy()  # Supprimer les anciens boutons

    # Ajouter un bouton pour chaque langue et afficher les phrases d'arrêt
    for language, code in supported_languages.items():
        stop_phrase = stop_phrases.get(code, "")
        button_text = f"{language} - Phrase d'arrêt : {stop_phrase}"
        button = tk.Button(window, text=button_text, command=lambda lang=language: change_language(lang))
        button.pack(pady=5)

def open_language_window():
    global window
    # Créer la fenêtre de changement de langue
    window = tk.Tk()
    window.title("Change Language")

    # Ajouter un bouton pour chaque langue et afficher les phrases d'arrêt
    update_buttons()

    # Placer la fenêtre au premier plan
    window.lift()  # Met la fenêtre Tkinter au premier plan
    window.attributes('-topmost', True)  # Assure que la fenêtre reste toujours au-dessus
    window.after(100, lambda: window.attributes('-topmost', False))  # Assure que la fenêtre ne reste pas constamment au-dessus

    # Démarrer la fenêtre
    window.mainloop()

# Initialiser le recognizer et le microphone
r = sr.Recognizer()
m = sr.Microphone()

# Ajuster le seuil de pause pour ignorer de petites pauses dans le discours
r.pause_threshold = 1  # Temps en secondes que le programme attend après une pause avant de traiter l'audio

print("Démarrage de la transcription en temps réel...")
print(f"Langue par défaut : {current_language}")
print(f"Commencez à parler")

# Démarrer l'écoute en arrière-plan
stop_listening = r.listen_in_background(m, callback)

# Attendre un moment pour que le microphone se stabilise et que l'ajustement pour le bruit ambiant soit effectué
time.sleep(1)  # Attendre quelques secondes pour donner du temps au microphone

# Ajuster pour le bruit ambiant
r.adjust_for_ambient_noise(m)

# Ouvrir la fenêtre pour changer de langue (dans un thread séparé pour ne pas bloquer l'écoute)
language_thread = threading.Thread(target=open_language_window)
language_thread.daemon = True
language_thread.start()

# Garder le programme actif pour la transcription continue
try:
    while not stop_flag:  # Boucle qui s'arrête si stop_flag est activé
        time.sleep(0.1)
except KeyboardInterrupt:
    stop_listening(wait_for_stop=False)
    print("Transcription arrêtée.")
    if window:
        window.quit()  # Fermer la fenêtre Tkinter quand l'utilisateur interrompt le programme


Démarrage de la transcription en temps réel...
Langue par défaut : fr-FR
Commencez à parler
Texte retranscrit (fr-FR): bonjour
Caractéristiques audio extraites :                                audspec_lengthL1norm_sma_range  \
start  end                                                      
0 days 0 days 00:00:14.080000                        1.116885   

                               audspec_lengthL1norm_sma_maxPos  \
start  end                                                       
0 days 0 days 00:00:14.080000                         0.665239   

                               audspec_lengthL1norm_sma_minPos  \
start  end                                                       
0 days 0 days 00:00:14.080000                         0.021413   

                               audspec_lengthL1norm_sma_quartile1  \
start  end                                                          
0 days 0 days 00:00:14.080000                            6.187626   

                               audsp

Texte retranscrit (fr-FR): stoppe stoppe
Phrase 'stoppe stoppe' détectée. Arrêt du programme.
Caractéristiques audio extraites :                                audspec_lengthL1norm_sma_range  \
start  end                                                      
0 days 0 days 00:00:03.904000                        0.361333   

                               audspec_lengthL1norm_sma_maxPos  \
start  end                                                       
0 days 0 days 00:00:03.904000                         0.629243   

                               audspec_lengthL1norm_sma_minPos  \
start  end                                                       
0 days 0 days 00:00:03.904000                          0.18799   

                               audspec_lengthL1norm_sma_quartile1  \
start  end                                                          
0 days 0 days 00:00:03.904000                             6.22606   

                               audspec_lengthL1norm_sma_quartile2  \
